<a href="https://colab.research.google.com/github/varun-sirpal10/emoji-predictor/blob/master/Emoji_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install emoji

     |████████████████████████████████| 51kB 2.0MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=f44eb04294964eace08794b630c81a89a2d5e6b8703c03fc112f37942bf8d55d
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [0]:
import emoji

In [0]:
emoji_dictionary = {"0":"\u2764\uFE0F",
                   "1":":baseball:",
                   "2":":grinning_face_with_big_eyes:",
                   "3":":disappointed_face:",
                   "4":":fork_and_knife:",
                   "5":":hundred_points:",
                   "6":":fire:",
                   "7":":face_blowing_a_kiss:",
                   "8":":chestnut:",
                   "9":":flexed_biceps:"
                   }

In [14]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴
💯
🔥
😘
🌰
💪


In [0]:
import pandas as pd
import numpy as np

In [0]:
train = pd.read_csv("drive/My Drive/train_emoji.csv",header=None)
test = pd.read_csv("drive/My Drive/test_emoji.csv",header=None)

In [18]:
train.head(n=5)

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [19]:
data = train.values
print(data.shape)

(132, 4)


In [0]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

In [21]:
for i in range(5):
    print(X_train[i],emoji.emojize(emoji_dictionary[str(Y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


In [0]:
f = open('glove.6B.50d.txt',encoding='utf-8')

In [0]:
embeddings_index = {}
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float')
    embeddings_index[word] = coefs
f.close()

In [24]:
emb_len = embeddings_index["eat"].shape[0]
print(emb_len)

50


In [25]:
def embedding_output(X):
    maxLen = 10
    embedding_out = np.zeros((X.shape[0],maxLen,emb_len))
    
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for ij in range(len(X[ix])):
            try:
                embedding_out[ix][ij] = embeddings_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij] = np.zeros((50,))
        
    return embedding_out

embedding_matrix_train = embedding_output(X_train)
embedding_matrix_test = embedding_output(X_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
embedding_matrix_train.shape

(132, 10, 50)

In [27]:
embedding_matrix_test.shape

(56, 10, 50)

In [28]:
from keras.models import Sequential
from keras.layers import *

Using TensorFlow backend.


In [66]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_7 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_6 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [0]:
from keras.utils import to_categorical

In [31]:
Y_train = to_categorical(Y_train,num_classes=5)
Y_test = to_categorical(Y_test,num_classes=5)

print(Y_train.shape)
#print(Y_test.shape)
print(Y_train[0])

(132, 5)
[0. 0. 0. 1. 0.]


In [67]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

#checkpoint = ModelCheckpoint(monitor='val_loss',verbose=False,save_best_only=True)
#earlystop = EarlyStopping(patience=10,monitor='val_acc')

hist = model.fit(embedding_matrix_train,Y_train,epochs=120,batch_size=64,shuffle=True,validation_split=0.2)

Train on 105 samples, validate on 27 samples
Epoch 1/120
105/105 [==============================] - 3s 30ms/step - loss: 1.5822 - acc: 0.2476 - val_loss: 1.6017 - val_acc: 0.2222
Epoch 2/120
105/105 [==============================] - 0s 595us/step - loss: 1.5631 - acc: 0.3143 - val_loss: 1.6014 - val_acc: 0.2222
Epoch 3/120
105/105 [==============================] - 0s 630us/step - loss: 1.5257 - acc: 0.3143 - val_loss: 1.6077 - val_acc: 0.2222
Epoch 4/120
105/105 [==============================] - 0s 640us/step - loss: 1.5119 - acc: 0.3810 - val_loss: 1.6191 - val_acc: 0.2222
Epoch 5/120
105/105 [==============================] - 0s 595us/step - loss: 1.4920 - acc: 0.4095 - val_loss: 1.6317 - val_acc: 0.2222
Epoch 6/120
105/105 [==============================] - 0s 621us/step - loss: 1.4557 - acc: 0.4095 - val_loss: 1.6465 - val_acc: 0.2593
Epoch 7/120
105/105 [==============================] - 0s 582us/step - loss: 1.4543 - acc: 0.4095 - val_loss: 1.6525 - val_acc: 0.2593
Epoch 8/120

In [68]:
pred = model.predict_classes(embedding_matrix_test)
print(pred)

[4 3 2 0 2 2 1 2 2 2 1 2 3 0 1 3 2 2 3 2 0 0 4 2 3 1 2 0 1 2 0 1 3 2 0 1 2
 3 2 2 2 0 0 1 2 3 3 2 2 1 3 0 3 2 3 0]


In [0]:
model.save("best_model.h5")

In [0]:
model.load_weights("my_best_model.h5")

In [69]:
model.evaluate(embedding_matrix_test,Y_test)

56/56 [==============================] - 0s 332us/step


[2.246293272290911, 0.6071428656578064]

In [0]:
pred = model.predict_classes(embedding_matrix_test)

In [72]:
for i in range(30):
  print(' '.join(X_test[i]))
  print(emoji.emojize(emoji_dictionary[str(np.argmax(Y_test[i]))]))
  print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😞
😞
he got a raise
😃
😃
she got me a present
❤️
❤️
ha ha ha it was so funny
😃
😃
he is a good friend
❤️
😃
I am upset
❤️
⚾
We had such a lovely dinner tonight
❤️
😃
where is the food
🍴
😃
Stop making this joke ha ha ha
😃
😃
where is the ball
⚾
⚾
work is hard
😞
😃
This girl is messing with me
😞
😞
are you serious ha ha
😃
❤️
Let us go play baseball
⚾
⚾
This stupid grader is not working
😞
😞
work is horrible
😞
😃
Congratulation for having a baby
😃
😃
stop messing around
😞
😞
any suggestions for dinner
🍴
😃
I love taking breaks
❤️
❤️
you brighten my day
😃
❤️
I boiled rice
🍴
🍴
she is a bully
😞
😃
Why are you feeling bad
😞
😞
I am upset
😞
⚾
I worked during my birthday
😞
😃
My grandmother is the love of my life
❤️
❤️
enjoy your break
😃
⚾
valentine day is near
❤️
😃
